In [1]:
import yfinance as yf
import pandas as pd
import time
import numpy as np

# List of company symbols
symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']
stock_data = []

for symbol in symbols:
    try:
        # Fetch stock data using yfinance
        ticker = yf.Ticker(symbol)
        profile = ticker.info

        stock_data.append({
            'Company Name': profile.get('longName', 'N/A'),
            'Symbol': symbol,
            'Current Price': profile.get('currentPrice', 'N/A'),
            'Market Cap (Intraday)': profile.get('marketCap', 'N/A'),
            'PE Ratio (TTM)': profile.get('trailingPE', 'N/A'),
            'P/B Ratio': profile.get('priceToBook', 'N/A'),
            'ROE': profile.get('returnOnEquity', 'N/A'),
            'Dividend Yield': profile.get('dividendYield', 'N/A')
        })
        
        # Sleep between requests to avoid rate limiting
        time.sleep(1)  # Adjust based on your needs

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        stock_data.append({
            'Company Name': 'N/A',
            'Symbol': symbol,
            'Current Price': 'N/A',
            'Market Cap (Intraday)': 'N/A',
            'PE Ratio (TTM)': 'N/A',
            'P/B Ratio': 'N/A',
            'ROE': 'N/A',
            'Dividend Yield': 'N/A'
        })

# Convert to DataFrame for better visualization
df = pd.DataFrame(stock_data)

# Convert relevant columns to numeric (if possible) to avoid comparison errors
df['PE Ratio (TTM)'] = pd.to_numeric(df['PE Ratio (TTM)'], errors='coerce')
df['P/B Ratio'] = pd.to_numeric(df['P/B Ratio'], errors='coerce')
df['ROE'] = pd.to_numeric(df['ROE'], errors='coerce')
df['Dividend Yield'] = pd.to_numeric(df['Dividend Yield'], errors='coerce')

# Assign weights for classification
weights = {
    'PE Ratio (TTM)': 0.25,
    'P/B Ratio': 0.25,
    'ROE': 0.25,
    'Dividend Yield': 0.25
}

# Function to classify stocks based on parameters
def classify_stock(row):
    classification_values = {'Undervalued': 1, 'Normal Valued': 0, 'Overvalued': -1}

    # Classify based on PE Ratio
    if row['PE Ratio (TTM)'] < 8:
        pe_category = 'Undervalued'
    elif 8 <= row['PE Ratio (TTM)'] <= 15:
        pe_category = 'Normal Valued'
    else:
        pe_category = 'Overvalued'

    # Classify based on P/B Ratio
    if row['P/B Ratio'] and row['P/B Ratio'] < 1:
        pb_category = 'Undervalued'
    elif row['P/B Ratio'] and 1 <= row['P/B Ratio'] <= 3:
        pb_category = 'Normal Valued'
    else:
        pb_category = 'Overvalued'
    
    # Classify based on ROE
    if row['ROE'] and row['ROE'] > 12:
        roe_category = 'Undervalued'
    elif row['ROE'] and 8 <= row['ROE'] <= 12:
        roe_category = 'Normal Valued'
    else:
        roe_category = 'Overvalued'
    
    # Classify based on Dividend Yield
    if pd.notnull(row['Dividend Yield']) and row['Dividend Yield'] > 3.5:
        dividend_category = 'Undervalued'
    elif pd.notnull(row['Dividend Yield']) and 2 <= row['Dividend Yield'] <= 3.5:
        dividend_category = 'Normal Valued'
    else:
        dividend_category = 'Overvalued'

    # Calculate weighted average for stock classification
    categories = [pe_category, pb_category, roe_category, dividend_category]
    weighted_average = np.average([classification_values[cat] for cat in categories], weights=[weights['PE Ratio (TTM)'], weights['P/B Ratio'], weights['ROE'], weights['Dividend Yield']])

    # Return the classification of the stock based on weighted average
    if weighted_average >= 0.6:
        final_category = 'Undervalued'
    elif weighted_average <= -0.4:
        final_category = 'Overvalued'
    else:
        final_category = 'Normal Valued'

    return pe_category, pb_category, roe_category, dividend_category, final_category

# Apply classification to each stock and calculate weighted average
df[['PE Category', 'PB Category', 'ROE Category', 'Dividend Category', 'Overall Category']] = df.apply(classify_stock, axis=1, result_type='expand')

# Function to apply conditional formatting based on category
def apply_color(val):
    color_map = {
        'Undervalued': 'background-color: lightgreen',
        'Normal Valued': 'background-color: lightyellow',
        'Overvalued': 'background-color: lightcoral'
    }
    return color_map.get(val, '')

# Apply conditional formatting to columns
styled_df = df.style.applymap(apply_color, subset=['PE Category', 'PB Category', 'ROE Category', 'Dividend Category'])

# Display the styled DataFrame
styled_df

/tmp/ipykernel_67553/3887816852.py:123: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(apply_color, subset=['PE Category', 'PB Category', 'ROE Category', 'Dividend Category'])


,Company Name,Symbol,Current Price,Market Cap (Intraday),PE Ratio (TTM),P/B Ratio,ROE,Dividend Yield,PE Category,PB Category,ROE Category,Dividend Category,Overall Category
0,Apple Inc.,AAPL,198.150000,2976629129216,31.452380,44.648487,1.365200,0.530000,Overvalued,Overvalued,Overvalued,Overvalued,Overvalued
1,Alphabet Inc.,GOOGL,157.140000,1929946333184,19.544777,5.902637,0.329080,0.550000,Overvalued,Overvalued,Overvalued,Overvalued,Overvalued
2,Microsoft Corporation,MSFT,388.450000,2887729545216,31.351896,9.541412,0.342910,0.920000,Overvalued,Overvalued,Overvalued,Overvalued,Overvalued
3,"Tesla, Inc.",TSLA,252.310000,811560140800,123.681370,11.128704,0.104200,nan,Overvalued,Overvalued,Overvalued,Overvalued,Overvalued
